In [3]:
import pandas as pd
df = pd.read_csv("/Users/alyssaesca/Downloads/expenditures_and_pit.csv")
data.head()

,City,Year,Total PEH,Unsheltered PEH,Population,"PEH Per 100,000","Unsheltered Per 100,000",Grantee,Program,Amount,ExpenditureType
0,El Cajon,2014,711,191,103230,688.753269,185.023733,East County Transitional Living Center,emergency shelter,75000.0,Crisis Management
1,El Cajon,2014,711,191,103230,688.753269,185.023733,East County Transitional Living Center,emergency shelter,75000.0,Crisis Management
2,El Cajon,2015,321,218,103527,310.064041,210.573087,East County Transitional Living Center,emergency shelter,120000.0,Crisis Management
3,El Cajon,2015,321,218,103527,310.064041,210.573087,East County Transitional Living Center,emergency shelter,120000.0,Crisis Management
4,San Marcos,2015,99,44,94932,104.285173,46.348966,North County Lifeline,housing assistance,21600.0,Prevention


In [4]:
# Grouping by City, Year, and Program, then aggregate the total amount spent
pivoted_df = df.groupby(['City', 'Year', 'Program'])['Amount'].sum().reset_index()

pivoted_df = pivoted_df.pivot(index=['City', 'Year'], columns='Program', values='Amount').fillna(0).reset_index()

In [6]:
pivoted_df.to_csv('/Users/alyssaesca/Downloads/pivoted_dataset.csv', index=False)

In [8]:
# noticed there were a lot of zeroes as some programs only occur once or twice throughout the entire dataset

# adding original and pivoted data sets for regression
original_df = pd.read_csv('/Users/alyssaesca/Downloads/expenditures_and_pit.csv')
pivoted_df = pd.read_csv('/Users/alyssaesca/Downloads/pivoted_dataset.csv')

merged_df = pd.merge(pivoted_df, original_df[['City', 'Year', 'Unsheltered PEH']], on=['City', 'Year'], how='left')

print(merged_df.head())

#added Unsheltered PEH to pivoted dataset

       City  Year  bridge to housing network  emergency shelter  \
0  Carlsbad  2017                        0.0            14896.0   
1  Carlsbad  2018                        0.0                0.0   
2  Carlsbad  2018                        0.0                0.0   
3  Carlsbad  2018                        0.0                0.0   
4  Carlsbad  2018                        0.0                0.0   

   family reunification program  flexible funds  food and nutrition  \
0                           0.0             0.0                 0.0   
1                           0.0             0.0                 0.0   
2                           0.0             0.0                 0.0   
3                           0.0             0.0                 0.0   
4                           0.0             0.0                 0.0   

   homeless services  homelessness prevention  homeshare program  ...  \
0                0.0                      0.0                0.0  ...   
1                0.0    

In [11]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Lasso
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

# List of columns to drop if they exist in the merged dataframe
columns_to_drop = ['City', 'Year', 'Total PEH', 'Population', 'PEH Per 100,000', 'Unsheltered Per 100,000']

# Drop columns only if they are in the dataframe
X = merged_df.drop(columns=[col for col in columns_to_drop if col in merged_df.columns])

# Set response variable and independent variables
y = merged_df['Unsheltered PEH']

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Apply Lasso Regression
lasso = Lasso(alpha=0.1)  
lasso.fit(X_train, y_train)

y_pred = lasso.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("Mean Squared Error:", mse)
print("R^2 Score:", r2)

feature_importance = pd.Series(lasso.coef_, index=X.columns)
print("Feature Importance:\n", feature_importance)


Mean Squared Error: 3.3859369367530574
R^2 Score: 0.999994015870902
Feature Importance:
 bridge to housing network               1.248526e+00
emergency shelter                      -0.000000e+00
family reunification program            1.251843e+01
flexible funds                         -9.451026e-03
food and nutrition                      6.046174e-02
homeless services                       0.000000e+00
homelessness prevention                 0.000000e+00
homeshare program                      -0.000000e+00
housing assistance                      3.009696e-01
housing navigation services             4.135368e+00
housing stability services             -0.000000e+00
motel voucher                           1.873554e-01
neighborhood revitalization services   -6.340011e-01
opening doors program                  -1.142107e-01
outreach                                2.505776e-01
project h.o.p.e.                       -0.000000e+00
rapid re housing                        1.010079e-01
rental ass